## Importing required libraries

In [73]:
from os import stat
from networkx.algorithms.components.connected import is_connected
from networkx.classes.function import neighbors
from networkx.linalg.algebraicconnectivity import fiedler_vector
import scipy as sp
import networkx as nx
from scipy.io import mmread
from scipy.sparse.coo import coo_matrix
from scipy.sparse.linalg import eigs
from numpy import linalg as LA
import numpy as np
import csv
import statistics

## Creating Graph object

In [74]:
class Graph:
    def __init__(self, sparse):
        self.graph = nx.from_scipy_sparse_matrix(sparse)
        self.adj = nx.adjacency_matrix(self.graph)
        self.laplacian = nx.laplacian_matrix(self.graph)
    def degree_centrality(self):
        return nx.degree_centrality(self.graph)
    def closeness_centrality(self):
        return nx.closeness_centrality(self.graph)
    def closeness_centrality_node(self, node):
        return nx.closeness_centrality(self.graph, node)
    def betweenness_centrality(self):
        return nx.betweenness_centrality(self.graph, k = min(self.graph.number_of_nodes() , 500))
    def eigenvector_centrality(self):
        
        return nx.eigenvector_centrality(self.graph)
    def is_connected(self):
        return nx.is_connected(self.graph)
    def lfvc(self):
        if (not self.is_connected()):
            return "Not possible"
        fiedler_vector = nx.fiedler_vector(self.graph)
        lfvclist = []
        for i in self.graph.nodes(data = True):
            lfvcthis = 0
            for j in self.graph.neighbors(i[0]):
                lfvcthis += (fiedler_vector[j]-fiedler_vector[i[0]])*(fiedler_vector[j]-fiedler_vector[i[0]])
            lfvclist.append(lfvcthis)
        return lfvclist
    def lfvc_node(self, node):
        if (not self.is_connected()):
            return "Not possible"
        lfvcthis = 0
        nodes = list(self.graph.nodes(data = True))
        n = nodes[node]
        fiedler_vector = self.eigenvector_atindex(1)
        fiedler = fiedler_vector[n[0]]
        for j in self.graph.neighbors(n[0]):
            lfvcthis += (fiedler_vector[j]-fiedler)*(fiedler_vector[j]-fiedler)
        return lfvcthis
    def neighbourhood_hopset(self, index, k = 10):
        nbrs = set([index])
        for l in range(k):
            nbrs = set((nbr for n in nbrs for nbr in self.graph[n]))
        return len(nbrs)
    def clustering_coefficient(self):
        return nx.clustering(self.graph)
    def clustering_coefficient_node(self, node):
        return nx.clustering(self.graph, node)
    def ego_centrality_node(self, node):
        g = nx.ego_graph(self.graph, node)
        nodes = list(g.nodes(data = True))
        n = node
        for i in nodes:
            if i[0] == node:
                n = i
                break
        centrality =  nx.betweenness_centrality(g)
        return centrality[node]
    def nodes_of_interest(self):
        l = list(nx.degree_centrality(self.graph))
        mean = statistics.mean(l)
        median = statistics.median_high(l)
        closest_mean = min(l, key = lambda x:abs(x-mean))
        max_value = max(l)
        min_value = min(l)
        return l.index(median), l.index(closest_mean), l.index(min_value), l.index(max_value)
    def eigenvector_atindex(self, a):
        eig_values, eig_vectors = eigs(internet)
        vector_pos = np.where(eig_values.real == np.sort(eig_values.real)[a])[0][0]
        vector = np.transpose(eig_vectors)[vector_pos]
        return vector.real

## Initialising and creating instances of graph object using different *.mtx files

In [75]:
# karate = mmread('../assets/S_soc-karate.mtx')
# webedu = mmread('../assets/M_web-edu.mtx')
# internet = mmread('../assets/L_tech-internet-as.mtx')

In [76]:
karate = mmread('soc-karate.mtx')
webedu = mmread('web-edu.mtx')
internet = mmread('tech-internet-as.mtx')

In [77]:
G1 = Graph(karate)
G2 = Graph(webedu)
G3 = Graph(internet)
display(G2.graph)
display(G3.graph)
print(("-"*50)+"Graphs made"+("-"*50))

--------------------------------------------------Graphs made--------------------------------------------------


In [78]:
G1.graph.size()

78

In [79]:
# Checking if instantiated graphs are connected

c1 = G1.is_connected()
c2 = G2.is_connected()
c3 = G3.is_connected()
print(f'G1 is connected: {c1}')
print(f'G2 is connected: {c2}')
print(f'G3 is connected: {c3}')

G1 is connected: True
G2 is connected: True
G3 is connected: True


## Finding Centralities

In [80]:
# EGO centrality
# print(G.ego_centrality_node(4))
# print("ego graph made")

In [81]:
# Finding lfvc node

lfvc1 = G1.lfvc_node(0)
lfvc2 = G2.lfvc_node(0)
# lfvc3 = G3.lfvc_node(0)
print(lfvc1)
print(lfvc2)
# print(lfvc3)

4.579948275797112e-05
2.999984025334377e-05


In [82]:
# Finding nodes of interest
print("Nodes of interest: ")
print(G1.nodes_of_interest())
print(G2.nodes_of_interest())
print(G3.nodes_of_interest())

Nodes of interest: 
(17, 16, 0, 33)
(1515, 1515, 0, 3030)
(20082, 20081, 0, 40163)


In [83]:
# Finding Centralities of smallest size graph, i.e. soc-karate

print("soc-karate :")
dc1 = G1.degree_centrality()
cc1 = G1.closeness_centrality()
bc1 = G1.betweenness_centrality()
ec1 = G1.eigenvector_centrality()
clc1 = G1.clustering_coefficient_node(0)
lfvc_val1 = G1.lfvc()
nhc1 = G1.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val1)
print(("-"*100))
print("degree centrality")
print(dc1)
print(("-"*100))
print("closeness centrality")
print(cc1)
print(("-"*100))
print("betweenness centrality")
print(bc1)
print(("-"*100))
print("eigenvector centrality")
print(ec1)
print(("-"*100))
print("Clusters of node 1")
print(clc1)
print(("-"*100))
print("neighbouring hopset")
print(nhc1)
print(("-"*100))


soc-karate :
----------------------------------------------------------------------------------------------------
lfvc
[0.2796644833360526, 0.031092730814211218, 0.06907300471272876, 0.014187972097852893, 0.03127549659853135, 0.056109398820060774, 0.05610939881905091, 0.009428901093859616, 0.03882565619806071, 0.005522978084165531, 0.031275496598491166, 0.0097723904840769, 0.002919005726748539, 0.031153835033945865, 0.0029727095351063117, 0.002972709535084597, 0.01961710726065581, 0.0036113936112126023, 0.002972709535073892, 0.028033824611904557, 0.0029727095350694393, 0.0036113936112382294, 0.0029727095350913823, 0.002952581755071063, 0.0036516741381748038, 0.003960776899744731, 0.005030760024818684, 0.012414442140584403, 0.005751480703857659, 0.004289527923188955, 0.018948645138262908, 0.05270747274383402, 0.029321908678190903, 0.06187316806881047]
----------------------------------------------------------------------------------------------------
degree centrality
{0: 0.484848484848

In [84]:
# Finding Centralities of medium size graph, i.e. web-edu

print("web-edu :")
dc2 = G2.degree_centrality()
cc2 = G2.closeness_centrality()
bc2 = G2.betweenness_centrality()
ec2 = G2.eigenvector_centrality()
clc2 = G2.clustering_coefficient_node(0)
lfvc_val2 = G2.lfvc()
nhc2 = G2.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val2)
print(("-"*100))
print("degree centrality")
print(dc2)
print(("-"*100))
print("closeness centrality")
print(cc2)
print(("-"*100))
print("betweenness centrality")
print(bc2)
print(("-"*100))
print("eigenvector centrality")
print(ec2)
print(("-"*100))
print("Clusters of node 1")
print(clc2)
print(("-"*100))
print("neighbouring hopset")
print(nhc2)
print(("-"*100))

web-edu :


KeyboardInterrupt: 

In [85]:
# Finding Centralities of largest size graph, i.e. tech-internet-as

print("tech-internet-as :")
# dc3 = G3.degree_centrality()
# cc3 = G3.closeness_centrality()
# bc3 = G3.betweenness_centrality()
# ec3 = G3.eigenvector_centrality()
# clc3 = G3.clustering_coefficient_node(0)
lfvc_val3 = G3.lfvc_node(0)
nhc3 = G3.neighbourhood_hopset(0,2)
print(("-"*100))
print("lfvc")
print(lfvc_val3)
# print(("-"*100))
# print("degree centrality")
# print(dc3)
# print(("-"*100))
# print("closeness centrality")
# print(cc3)
# print(("-"*100))
# print("betweenness centrality")
# print(bc3)
# print(("-"*100))
# print("eigenvector centrality")
# print(ec3)
# print(("-"*100))
# print("Clusters of node 1")
# print(clc3)
# print(("-"*100))
# print("neighbouring hopset")
# print(nhc3)
# print(("-"*100))

tech-internet-as :
----------------------------------------------------------------------------------------------------
lfvc
7.920438705271443e-05
